# Simulation 06 -- Sparse Region Stress Test

## Question
**In regions with very few sites (e.g., Rural Montana: ~15 sites), does the pipeline still produce usable FMV estimates?**

## Why This Matters
The Verizon portfolio spans the full urban-rural spectrum. Dense metro areas (NYC, LA, Chicago) may have hundreds of sites, providing rich training data for HDBSCAN clustering and supervised models. But rural regions may have only 10-30 sites. At this scale:

- **Clustering may fail** -- HDBSCAN needs a minimum cluster size; 15 sites might all become noise
- **Supervised models may overfit** -- XGBoost/QRF trained on 5-10 CORE sites will memorise, not generalise
- **Confidence bands may be misleadingly narrow** -- small samples can produce tight quantile bands that look confident but are unreliable

## Approach
1. Categorise regions by site density (Dense / Moderate / Sparse)
2. Run the full pipeline and measure MAPE by density tier
3. Check QRF confidence band widths by density
4. Test three sparse-region fallback strategies
5. Find the minimum viable region size via subsampling

## Output
A summary table with MAPE and confidence metrics by region, plus recommendations for handling sparse regions.

In [ ]:
# ============================================================
# Imports & Configuration
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import median_abs_deviation
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error, r2_score

import xgboost as xgb

# HDBSCAN -- try standalone package first, fall back to sklearn
try:
    from hdbscan import HDBSCAN
    print('Using standalone hdbscan package')
except ImportError:
    from sklearn.cluster import HDBSCAN
    print('Using sklearn HDBSCAN')

# Quantile Random Forest
try:
    from quantile_forest import RandomForestQuantileRegressor
    HAS_QRF = True
    print('Using quantile_forest package')
except ImportError:
    from sklearn.ensemble import RandomForestRegressor
    HAS_QRF = False
    print('quantile_forest not available -- falling back to sklearn RF')

sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 120
np.random.seed(42)

print('\nAll imports successful.')

## Load Synthetic Data

In [ ]:
df = pd.read_parquet('../data/synthetic/synthetic_5k_sites.parquet')
print(f'Loaded {len(df):,} sites  |  {df.shape[1]} columns')
print(f'Unique metro regions: {df["metro_region"].nunique()}')
print(f'Rent range: ${df["current_monthly_rent"].min():,.0f} - ${df["current_monthly_rent"].max():,.0f}')
print(f'True FMV range: ${df["true_fmv"].min():,.0f} - ${df["true_fmv"].max():,.0f}')
df.head(3)

## Define Feature Lists

In [ ]:
X_MARKET = [
    # Market / Demographics
    'census_population_density', 'census_population_3km',
    'census_median_income', 'census_median_home_value',
    'census_median_rent', 'census_vacancy_rate',
    'urban_rural_class', 'underlying_land_value_psf',
    'hud_safmr', 'property_tax_rate',
    # Topography / Geography
    'topography_class', 'terrain_elevation_variance_m',
    'poi_density_1km', 'building_density_500m',
    'distance_to_highway_km', 'ground_elevation_ft',
    'unemployment_rate_local',
    # Competition / Supply
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm',
    'competitor_tower_distance_km', 'tower_per_capita',
    'scarcity_index', 'landlord_concentration',
    'composite_hazard_score',
    # Property Characteristics
    'tower_height_ft', 'lease_area_sqft', 'parcel_acreage',
    'construction_age', 'site_access_ordinal',
    # Zoning / Regulatory
    'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft',
    'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
]

print(f'X_MARKET feature count: {len(X_MARKET)}')
missing = [f for f in X_MARKET if f not in df.columns]
if missing:
    print(f'WARNING: Missing columns: {missing}')
else:
    print('All features present in dataset.')

---
## Step 1: Categorise Regions by Density

We split all metro regions into three tiers based on their site count:
- **Dense**: Top 10 regions by site count (metros like NYC, LA, Chicago)
- **Moderate**: Next 10 regions
- **Sparse**: Bottom 10 regions (rural areas with fewest sites)

In [ ]:
# Count sites per region
region_counts = df['metro_region'].value_counts().reset_index()
region_counts.columns = ['metro_region', 'site_count']
region_counts = region_counts.sort_values('site_count', ascending=False).reset_index(drop=True)

n_regions = len(region_counts)
print(f'Total regions: {n_regions}')
print(f'Total sites: {region_counts["site_count"].sum():,}')

# Assign density tiers
top_10 = region_counts.head(10)['metro_region'].tolist()
next_10 = region_counts.iloc[10:20]['metro_region'].tolist()
bottom_10 = region_counts.tail(10)['metro_region'].tolist()

def assign_tier(region):
    if region in top_10:
        return 'Dense'
    elif region in next_10:
        return 'Moderate'
    else:
        return 'Sparse'

df['density_tier'] = df['metro_region'].apply(assign_tier)
region_counts['density_tier'] = region_counts['metro_region'].apply(assign_tier)

# Print distribution table
print('\n' + '=' * 65)
print(f'{"Region":25s} {"Sites":>7s} {"Tier":>10s}')
print('-' * 65)
for _, row in region_counts.iterrows():
    print(f"{row['metro_region']:25s} {row['site_count']:7d} {row['density_tier']:>10s}")

# Tier summary
print('\n' + '=' * 65)
tier_summary = df.groupby('density_tier').agg(
    n_sites=('site_id', 'count'),
    n_regions=('metro_region', 'nunique'),
    mean_rent=('current_monthly_rent', 'mean'),
    mean_fmv=('true_fmv', 'mean')
).reindex(['Dense', 'Moderate', 'Sparse'])
tier_summary['pct_of_portfolio'] = (tier_summary['n_sites'] / len(df) * 100).round(1)
print('\nTier Summary:')
print(tier_summary.to_string())

---
## Step 2: Run Full Pipeline on All Data

We run the complete HDBSCAN -> pseudo-label -> XGBoost+QRF -> classify pipeline on the entire dataset, then measure performance broken down by density tier.

In [ ]:
def run_full_pipeline(df_in, features, label='Baseline'):
    """
    Full pipeline: HDBSCAN -> pseudo-label -> XGBoost+QRF -> predict.
    Returns the dataframe with added prediction and confidence columns.
    """
    df_pipe = df_in.copy()
    
    # --- Stage 1: Clustering ---
    X = df_pipe[features].fillna(df_pipe[features].median()).values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    clusterer = HDBSCAN(min_cluster_size=20, min_samples=5)
    df_pipe['cluster'] = clusterer.fit_predict(X_scaled)
    
    n_clusters = len(set(df_pipe['cluster'])) - (1 if -1 in df_pipe['cluster'].values else 0)
    n_noise = (df_pipe['cluster'] == -1).sum()
    print(f'  [{label}] Clusters: {n_clusters}, Noise: {n_noise} ({n_noise/len(df_pipe)*100:.1f}%)')
    
    # --- Stage 2: Pseudo-labeling ---
    df_pipe['pseudo_label'] = 'EDGE'
    for c in sorted(df_pipe['cluster'].unique()):
        if c == -1:
            df_pipe.loc[df_pipe['cluster'] == c, 'pseudo_label'] = 'OUTLIER'
            continue
        mask = df_pipe['cluster'] == c
        rents = df_pipe.loc[mask, 'current_monthly_rent'].values
        med = np.median(rents)
        mad = median_abs_deviation(rents, scale='normal')
        mad = max(mad, 1.0)
        deviations = np.abs(rents - med) / mad
        idx = df_pipe.index[mask]
        df_pipe.loc[idx[deviations <= 1.0], 'pseudo_label'] = 'CORE'
        df_pipe.loc[idx[deviations > 2.0], 'pseudo_label'] = 'OUTLIER'
    
    n_core = (df_pipe['pseudo_label'] == 'CORE').sum()
    print(f'  [{label}] CORE sites: {n_core} ({n_core/len(df_pipe)*100:.1f}%)')
    
    # --- Stage 3: Supervised Model ---
    core_mask = df_pipe['pseudo_label'] == 'CORE'
    X_train = df_pipe.loc[core_mask, features].fillna(df_pipe[features].median()).values
    y_train = df_pipe.loc[core_mask, 'current_monthly_rent'].values
    X_all = df_pipe[features].fillna(df_pipe[features].median()).values
    
    # XGBoost
    xgb_model = xgb.XGBRegressor(
        n_estimators=300, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8, min_child_weight=5,
        reg_alpha=0.1, reg_lambda=1.0, random_state=42, verbosity=0
    )
    xgb_model.fit(X_train, y_train)
    df_pipe['pred_xgb'] = xgb_model.predict(X_all)
    
    # QRF
    if HAS_QRF:
        qrf_model = RandomForestQuantileRegressor(
            n_estimators=300, min_samples_leaf=5, random_state=42, n_jobs=-1
        )
        qrf_model.fit(X_train, y_train)
        df_pipe['pred_qrf_50'] = qrf_model.predict(X_all, quantiles=0.5)
        df_pipe['pred_qrf_25'] = qrf_model.predict(X_all, quantiles=0.25)
        df_pipe['pred_qrf_75'] = qrf_model.predict(X_all, quantiles=0.75)
    else:
        rf_model = RandomForestRegressor(
            n_estimators=300, min_samples_leaf=5, random_state=42, n_jobs=-1
        )
        rf_model.fit(X_train, y_train)
        pred_rf = rf_model.predict(X_all)
        df_pipe['pred_qrf_50'] = pred_rf
        # Approximate quantile bands using individual tree predictions
        tree_preds = np.array([t.predict(X_all) for t in rf_model.estimators_])
        df_pipe['pred_qrf_25'] = np.percentile(tree_preds, 25, axis=0)
        df_pipe['pred_qrf_75'] = np.percentile(tree_preds, 75, axis=0)
    
    # Ensemble
    df_pipe['predicted_fmv'] = (df_pipe['pred_xgb'] + df_pipe['pred_qrf_50']) / 2
    df_pipe['qrf_band_width'] = df_pipe['pred_qrf_75'] - df_pipe['pred_qrf_25']
    df_pipe['qrf_band_pct'] = df_pipe['qrf_band_width'] / df_pipe['predicted_fmv'].clip(lower=1)
    
    # --- Stage 4: Classification ---
    df_pipe['classification'] = 'AT_MARKET'
    df_pipe.loc[df_pipe['current_monthly_rent'] > df_pipe['pred_qrf_75'], 'classification'] = 'OVERPAYING'
    df_pipe.loc[df_pipe['current_monthly_rent'] < df_pipe['pred_qrf_25'], 'classification'] = 'UNDERPAYING'
    
    # Metrics vs true FMV
    df_pipe['abs_pct_error'] = np.abs(df_pipe['predicted_fmv'] - df_pipe['true_fmv']) / df_pipe['true_fmv']
    mape = df_pipe['abs_pct_error'].mean() * 100
    print(f'  [{label}] Overall MAPE vs true FMV: {mape:.2f}%')
    
    return df_pipe


print('Running full pipeline on all data...')
print()
df_result = run_full_pipeline(df, X_MARKET, label='Full')

---
## Step 3: MAPE by Region Density

How does FMV estimation accuracy vary across dense, moderate, and sparse regions?

In [ ]:
# MAPE by density tier
tier_metrics = df_result.groupby('density_tier').agg(
    n_sites=('site_id', 'count'),
    mape=('abs_pct_error', lambda x: x.mean() * 100),
    median_mape=('abs_pct_error', lambda x: x.median() * 100),
    mae=('abs_pct_error', lambda x: (x * df_result.loc[x.index, 'true_fmv']).mean()),
    mean_band_pct=('qrf_band_pct', 'mean'),
).reindex(['Dense', 'Moderate', 'Sparse'])

print('MAPE by Density Tier')
print('=' * 70)
print(f'{"Tier":12s} {"Sites":>7s} {"MAPE":>8s} {"Med MAPE":>10s} {"Band Width":>12s}')
print('-' * 70)
for tier, row in tier_metrics.iterrows():
    print(f'{tier:12s} {row["n_sites"]:7.0f} {row["mape"]:7.1f}% {row["median_mape"]:9.1f}% '
          f'{row["mean_band_pct"]*100:11.1f}%')

# Per-region MAPE
region_metrics = df_result.groupby('metro_region').agg(
    n_sites=('site_id', 'count'),
    mape=('abs_pct_error', lambda x: x.mean() * 100),
    mean_band_pct=('qrf_band_pct', 'mean'),
    density_tier=('density_tier', 'first')
).sort_values('n_sites', ascending=False)

print(f'\nPer-Region Breakdown:')
print(f'{"Region":25s} {"Sites":>7s} {"MAPE":>8s} {"Band %":>8s} {"Tier":>10s}')
print('-' * 65)
for region, row in region_metrics.iterrows():
    print(f'{region:25s} {row["n_sites"]:7.0f} {row["mape"]:7.1f}% '
          f'{row["mean_band_pct"]*100:7.1f}% {row["density_tier"]:>10s}')

In [ ]:
# ---- Plot: MAPE by density tier + Scatter: MAPE vs site count ----
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Left: Bar chart of MAPE by tier
ax = axes[0]
tier_order = ['Dense', 'Moderate', 'Sparse']
tier_colors = {'Dense': '#2ca02c', 'Moderate': '#ff7f0e', 'Sparse': '#d62728'}
mape_vals = [tier_metrics.loc[t, 'mape'] for t in tier_order]
bars = ax.bar(tier_order, mape_vals,
              color=[tier_colors[t] for t in tier_order],
              edgecolor='black', width=0.5)
for bar, val in zip(bars, mape_vals):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
            f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=12)
ax.set_ylabel('MAPE vs True FMV (%)', fontsize=12)
ax.set_title('MAPE by Region Density Tier', fontsize=13)
ax.axhline(15, color='gray', ls='--', alpha=0.5, label='Target: 15%')
ax.legend()
ax.set_ylim(0, max(mape_vals) * 1.4)

# Right: Scatter of MAPE per region vs site count
ax = axes[1]
for _, row in region_metrics.iterrows():
    c = tier_colors.get(row['density_tier'], 'gray')
    ax.scatter(row['n_sites'], row['mape'], c=c, s=80, edgecolors='black',
               linewidths=0.5, zorder=3)

# Annotate sparse regions
for region, row in region_metrics.iterrows():
    if row['density_tier'] == 'Sparse' or row['mape'] > tier_metrics.loc['Moderate', 'mape'] * 1.5:
        ax.annotate(region, xy=(row['n_sites'], row['mape']),
                    fontsize=7, ha='left', va='bottom',
                    xytext=(4, 2), textcoords='offset points')

ax.set_xlabel('Number of Sites in Region', fontsize=12)
ax.set_ylabel('MAPE vs True FMV (%)', fontsize=12)
ax.set_title('MAPE per Region vs Region Size', fontsize=13)
ax.axhline(15, color='gray', ls='--', alpha=0.5, label='Target: 15%')

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=tier_colors['Dense'], markersize=8, label='Dense'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=tier_colors['Moderate'], markersize=8, label='Moderate'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=tier_colors['Sparse'], markersize=8, label='Sparse'),
    Line2D([0], [0], color='gray', ls='--', alpha=0.5, label='Target: 15%'),
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

---
## Step 4: QRF Confidence Band Width by Density

The QRF produces prediction intervals (Q25-Q75). In sparse regions, we expect **wider** bands (more uncertainty). If the bands are **narrow** despite poor MAPE, the model is **overconfident** -- a dangerous situation where the pipeline claims high precision but is actually wrong.

In [ ]:
# Band width analysis by tier
band_analysis = df_result.groupby('density_tier').agg(
    mean_band_width=('qrf_band_width', 'mean'),
    mean_band_pct=('qrf_band_pct', 'mean'),
    median_band_pct=('qrf_band_pct', 'median'),
    mean_predicted_fmv=('predicted_fmv', 'mean'),
    mape=('abs_pct_error', lambda x: x.mean() * 100),
).reindex(['Dense', 'Moderate', 'Sparse'])

# Check for overconfidence: narrow bands + high MAPE
band_analysis['mape_to_band_ratio'] = band_analysis['mape'] / (band_analysis['mean_band_pct'] * 100)

print('QRF Confidence Band Analysis')
print('=' * 85)
print(f'{"Tier":12s} {"Mean Band$":>10s} {"Band%":>8s} {"MAPE":>8s} {"MAPE/Band":>11s} {"Diagnosis"}')
print('-' * 85)
for tier, row in band_analysis.iterrows():
    ratio = row['mape_to_band_ratio']
    if ratio > 1.5:
        diag = 'OVERCONFIDENT'
    elif ratio > 1.0:
        diag = 'Slightly overconfident'
    else:
        diag = 'Well-calibrated'
    print(f'{tier:12s} ${row["mean_band_width"]:8,.0f} {row["mean_band_pct"]*100:7.1f}% '
          f'{row["mape"]:7.1f}% {ratio:10.2f}   {diag}')

print(f'\nInterpretation:')
print(f'  MAPE/Band > 1.5 → Model errors routinely exceed stated confidence bands → OVERCONFIDENT')
print(f'  MAPE/Band ~ 1.0 → Bands appropriately capture uncertainty → WELL-CALIBRATED')
print(f'  MAPE/Band < 0.7 → Bands are wider than actual errors → CONSERVATIVE (OK)')

In [ ]:
# ---- Plot: Band width vs MAPE by tier ----
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Band width % by tier
ax = axes[0]
x = np.arange(3)
width = 0.35
tiers = ['Dense', 'Moderate', 'Sparse']
mape_vals = [band_analysis.loc[t, 'mape'] for t in tiers]
band_vals = [band_analysis.loc[t, 'mean_band_pct'] * 100 for t in tiers]

bars1 = ax.bar(x - width/2, band_vals, width, label='Mean Band Width %',
               color='#1f77b4', edgecolor='black')
bars2 = ax.bar(x + width/2, mape_vals, width, label='MAPE %',
               color='#d62728', edgecolor='black')

ax.set_xticks(x)
ax.set_xticklabels(tiers, fontsize=11)
ax.set_ylabel('Percentage (%)', fontsize=11)
ax.set_title('Confidence Band Width vs MAPE by Density', fontsize=13)
ax.legend()

# Annotate overconfidence
for i, tier in enumerate(tiers):
    if band_analysis.loc[tier, 'mape_to_band_ratio'] > 1.5:
        ax.annotate('OVERCONFIDENT', xy=(i, max(mape_vals[i], band_vals[i])),
                    fontsize=8, color='red', fontweight='bold',
                    ha='center', va='bottom', xytext=(0, 5), textcoords='offset points')

# Right: Per-region scatter: band width vs MAPE
ax = axes[1]
for region, row in region_metrics.iterrows():
    c = tier_colors.get(row['density_tier'], 'gray')
    ax.scatter(row['mean_band_pct'] * 100, row['mape'], c=c, s=80,
               edgecolors='black', linewidths=0.5, zorder=3)

# Draw the y=x line (perfect calibration)
lim = max(region_metrics['mape'].max(), region_metrics['mean_band_pct'].max() * 100) * 1.1
ax.plot([0, lim], [0, lim], 'k--', alpha=0.3, label='Perfect calibration (MAPE=Band%)')
ax.set_xlabel('Mean QRF Band Width (%)', fontsize=11)
ax.set_ylabel('MAPE vs True FMV (%)', fontsize=11)
ax.set_title('Calibration Check: Band Width vs Actual Error', fontsize=13)
ax.legend()

# Annotate some sparse regions
for region, row in region_metrics.iterrows():
    if row['density_tier'] == 'Sparse':
        ax.annotate(region, xy=(row['mean_band_pct']*100, row['mape']),
                    fontsize=6, ha='left', xytext=(3, 2), textcoords='offset points')

plt.tight_layout()
plt.show()

---
## Step 5: Sparse Region Fallback Strategies

For regions where the standard pipeline produces unreliable results, we test three fallback strategies:

| Option | Strategy | Trade-off |
|--------|----------|----------|
| **A** | Merge sparse region into nearest moderate/dense region | Borrows from different market -- potentially inappropriate |
| **B** | Use cluster median as FMV (no supervised model) | Simple, transparent, but loses feature-level information |
| **C** | Flag as "insufficient data" | Honest, but leaves sites without an estimate |

In [ ]:
# Get baseline MAPE for sparse regions from the full pipeline run
sparse_mask = df_result['density_tier'] == 'Sparse'
sparse_baseline_mape = df_result.loc[sparse_mask, 'abs_pct_error'].mean() * 100
print(f'Sparse region baseline MAPE: {sparse_baseline_mape:.1f}%')
print(f'Sparse region sites: {sparse_mask.sum()}')
print()

In [ ]:
# ============================================================
# OPTION A: Merge sparse regions into nearest moderate/dense region
# ============================================================
print('=== Option A: Merge Sparse into Nearest Non-Sparse Region ===')

# Compute region centroids
region_centroids = df.groupby('metro_region')[['lat', 'lon']].mean()

# For each sparse region, find the nearest non-sparse region
non_sparse_regions = region_counts[region_counts['density_tier'] != 'Sparse']['metro_region'].tolist()
sparse_regions = region_counts[region_counts['density_tier'] == 'Sparse']['metro_region'].tolist()

merge_map = {}  # sparse_region -> target_region
for sr in sparse_regions:
    sr_centroid = region_centroids.loc[sr].values.reshape(1, -1)
    ns_centroids = region_centroids.loc[non_sparse_regions].values
    dists = cdist(sr_centroid, ns_centroids, metric='euclidean')[0]
    nearest_idx = np.argmin(dists)
    merge_map[sr] = non_sparse_regions[nearest_idx]
    print(f'  {sr:25s} -> {non_sparse_regions[nearest_idx]:25s} (dist={dists[nearest_idx]:.2f} deg)')

# Create merged dataset
df_merged = df.copy()
df_merged['metro_region_merged'] = df_merged['metro_region'].map(
    lambda r: merge_map.get(r, r)
)

# Re-run full pipeline on merged data
print('\nRunning pipeline on merged data...')
df_merged_result = run_full_pipeline(df_merged, X_MARKET, label='MergedA')

# Measure MAPE on originally-sparse sites only
option_a_mape = df_merged_result.loc[sparse_mask, 'abs_pct_error'].mean() * 100
print(f'\n  Option A MAPE (sparse sites): {option_a_mape:.1f}% (baseline: {sparse_baseline_mape:.1f}%)')

In [ ]:
# ============================================================
# OPTION B: Use cluster median as FMV (no supervised model)
# ============================================================
print('=== Option B: Cluster Median as FMV for Sparse Regions ===')

# For sparse sites, use the cluster median rent as FMV estimate
# (bypassing the XGBoost/QRF supervised model)
df_optb = df_result.copy()

# Compute cluster medians
cluster_medians = df_optb.groupby('cluster')['current_monthly_rent'].median()

# For sparse sites, replace predicted_fmv with cluster median
for idx in df_optb[sparse_mask].index:
    c = df_optb.loc[idx, 'cluster']
    if c == -1:
        # Noise point: use global median of sparse sites
        df_optb.loc[idx, 'predicted_fmv_b'] = df_optb.loc[sparse_mask, 'current_monthly_rent'].median()
    else:
        df_optb.loc[idx, 'predicted_fmv_b'] = cluster_medians.get(c, df_optb.loc[idx, 'predicted_fmv'])

# For non-sparse, keep original predictions
df_optb.loc[~sparse_mask, 'predicted_fmv_b'] = df_optb.loc[~sparse_mask, 'predicted_fmv']

# Compute MAPE for sparse sites
sparse_true = df_optb.loc[sparse_mask, 'true_fmv'].values
sparse_pred_b = df_optb.loc[sparse_mask, 'predicted_fmv_b'].values
option_b_mape = np.mean(np.abs(sparse_pred_b - sparse_true) / sparse_true) * 100
print(f'  Option B MAPE (sparse sites): {option_b_mape:.1f}% (baseline: {sparse_baseline_mape:.1f}%)')

In [ ]:
# ============================================================
# OPTION C: Flag sparse regions as "insufficient data"
# ============================================================
print('=== Option C: Flag Sparse Regions as Insufficient Data ===')

n_sparse_sites = sparse_mask.sum()
pct_affected = n_sparse_sites / len(df) * 100

print(f'  Sparse region sites: {n_sparse_sites}')
print(f'  Percentage of portfolio affected: {pct_affected:.1f}%')
print(f'  These sites would receive NO FMV estimate (flagged for manual review)')

# Also check by property value -- are these high-value sites?
sparse_rent_total = df_result.loc[sparse_mask, 'current_monthly_rent'].sum()
total_rent = df_result['current_monthly_rent'].sum()
pct_rent = sparse_rent_total / total_rent * 100
print(f'  Percentage of total monthly rent: {pct_rent:.1f}%')

if pct_affected < 5:
    print(f'\n  -> Small portfolio impact. Flagging is a viable conservative option.')
elif pct_affected < 15:
    print(f'\n  -> Moderate portfolio impact. Consider fallback A or B for most sites.')
else:
    print(f'\n  -> Large portfolio impact. Flagging alone is insufficient -- need fallback.')

In [ ]:
# ---- Plot: Fallback comparison ----
fig, ax = plt.subplots(figsize=(10, 5))

strategies = ['Baseline\n(Full Pipeline)', 'Option A\n(Merge Regions)',
              'Option B\n(Cluster Median)', 'Option C\n(Flag/No Estimate)']
mapes = [sparse_baseline_mape, option_a_mape, option_b_mape, np.nan]
colors = ['#1f77b4', '#2ca02c', '#ff7f0e', '#bdbdbd']

bars = ax.bar(strategies, [m if not np.isnan(m) else 0 for m in mapes],
              color=colors, edgecolor='black', width=0.5)

for bar, val in zip(bars, mapes):
    if not np.isnan(val):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
                f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=12)
    else:
        ax.text(bar.get_x() + bar.get_width()/2, 1,
                'N/A\n(No estimate)', ha='center', va='bottom', fontsize=10, color='gray')

ax.axhline(15, color='gray', ls='--', alpha=0.5, label='Target MAPE: 15%')
ax.axhline(25, color='red', ls='--', alpha=0.5, label='Sparse threshold: 25%')
ax.set_ylabel('MAPE vs True FMV (%)', fontsize=12)
ax.set_title('Sparse Region Fallback Strategy Comparison', fontsize=13)
ax.legend()
ax.set_ylim(0, max([m for m in mapes if not np.isnan(m)]) * 1.4)

plt.tight_layout()
plt.show()

# Determine winner
valid_mapes = {'Baseline': sparse_baseline_mape, 'Option A': option_a_mape, 'Option B': option_b_mape}
best_strategy = min(valid_mapes, key=valid_mapes.get)
print(f'\nBest fallback for sparse regions: {best_strategy} (MAPE={valid_mapes[best_strategy]:.1f}%)')

---
## Step 6: Minimum Viable Region Size Analysis

To find the "knee" where pipeline performance becomes acceptable, we artificially subsample from a large region and measure how MAPE degrades as region size shrinks.

We take the largest region and repeatedly subsample at sizes from 10 to 200, running the full pipeline each time.

In [ ]:
# Pick the largest region for subsampling
largest_region = region_counts.iloc[0]['metro_region']
largest_region_size = region_counts.iloc[0]['site_count']
print(f'Subsampling from: {largest_region} ({largest_region_size} sites)')

# Get all sites from this region
region_mask = df['metro_region'] == largest_region
df_region = df[region_mask].copy()

# True FMV for later evaluation
region_true_fmv = df_region['true_fmv'].values

# Test sizes: 10 to min(200, region_size)
max_test_size = min(200, largest_region_size)
test_sizes = list(range(10, max_test_size + 1, 10))
if test_sizes[-1] != max_test_size:
    test_sizes.append(max_test_size)

N_REPEATS = 5  # average over multiple random subsamples

size_results = []
for size in test_sizes:
    mapes_at_size = []
    for rep in range(N_REPEATS):
        # Random subsample
        sample_idx = np.random.choice(len(df_region), size=min(size, len(df_region)), replace=False)
        df_sub = df_region.iloc[sample_idx].copy().reset_index(drop=True)
        
        # Prepare features
        X_sub = df_sub[X_MARKET].fillna(df_sub[X_MARKET].median()).values
        scaler = StandardScaler()
        X_sub_scaled = scaler.fit_transform(X_sub)
        
        # Clustering (adjust min_cluster_size for small samples)
        min_cs = max(5, min(20, size // 4))
        min_s = max(2, min(5, size // 6))
        try:
            labels = HDBSCAN(min_cluster_size=min_cs, min_samples=min_s).fit_predict(X_sub_scaled)
        except Exception:
            labels = np.zeros(len(df_sub), dtype=int)
        
        # Pseudo-labeling
        pseudo = np.full(len(df_sub), 'EDGE', dtype=object)
        for c in sorted(set(labels)):
            if c == -1:
                pseudo[labels == c] = 'OUTLIER'
                continue
            mask = labels == c
            rents = df_sub.loc[mask, 'current_monthly_rent'].values
            med = np.median(rents)
            mad = max(median_abs_deviation(rents, scale='normal'), 1.0)
            devs = np.abs(rents - med) / mad
            pseudo[(labels == c) & (devs <= 1.0)] = 'CORE'
            pseudo[(labels == c) & (devs > 2.0)] = 'OUTLIER'
        
        core_mask = pseudo == 'CORE'
        n_core = core_mask.sum()
        
        if n_core < 5:
            # Too few CORE sites -- use all non-outlier sites
            core_mask = pseudo != 'OUTLIER'
            n_core = core_mask.sum()
        
        if n_core < 3:
            # Absolute minimum -- use cluster median
            pred = np.full(len(df_sub), df_sub['current_monthly_rent'].median())
        else:
            # Train XGBoost
            X_train = df_sub.loc[core_mask, X_MARKET].fillna(df_sub[X_MARKET].median()).values
            y_train = df_sub.loc[core_mask, 'current_monthly_rent'].values
            X_pred = df_sub[X_MARKET].fillna(df_sub[X_MARKET].median()).values
            
            model = xgb.XGBRegressor(
                n_estimators=100, learning_rate=0.1, max_depth=4,
                subsample=0.8, min_child_weight=3, random_state=42+rep, verbosity=0
            )
            model.fit(X_train, y_train)
            pred = model.predict(X_pred)
        
        # MAPE vs true FMV
        true_fmv_sub = df_sub['true_fmv'].values
        mape = np.mean(np.abs(pred - true_fmv_sub) / true_fmv_sub) * 100
        mapes_at_size.append(mape)
    
    mean_mape = np.mean(mapes_at_size)
    std_mape = np.std(mapes_at_size)
    size_results.append({
        'region_size': size,
        'mean_mape': mean_mape,
        'std_mape': std_mape,
        'min_mape': np.min(mapes_at_size),
        'max_mape': np.max(mapes_at_size),
    })
    print(f'  Size={size:4d}: MAPE={mean_mape:.1f}% (±{std_mape:.1f}%)')

size_df = pd.DataFrame(size_results)

In [ ]:
# ---- Plot: MAPE vs region size ----
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(size_df['region_size'], size_df['mean_mape'], 'b-o', markersize=5, label='Mean MAPE')
ax.fill_between(size_df['region_size'],
                size_df['mean_mape'] - size_df['std_mape'],
                size_df['mean_mape'] + size_df['std_mape'],
                alpha=0.2, color='blue', label='±1 std')
ax.fill_between(size_df['region_size'],
                size_df['min_mape'], size_df['max_mape'],
                alpha=0.1, color='blue', label='Min-Max range')

# Target lines
ax.axhline(15, color='green', ls='--', alpha=0.6, label='Target: 15%')
ax.axhline(25, color='red', ls='--', alpha=0.6, label='Sparse threshold: 25%')

# Find the knee (first size where mean MAPE drops below 15%)
below_target = size_df[size_df['mean_mape'] <= 15]
if len(below_target) > 0:
    knee_size = below_target.iloc[0]['region_size']
    knee_mape = below_target.iloc[0]['mean_mape']
    ax.axvline(knee_size, color='purple', ls=':', alpha=0.6, label=f'Knee: {knee_size:.0f} sites')
    ax.scatter([knee_size], [knee_mape], color='purple', s=150, zorder=5, marker='*')
    print(f'Knee point: {knee_size:.0f} sites (MAPE={knee_mape:.1f}%)')
else:
    print(f'MAPE never drops below 15% in tested range. Minimum: {size_df["mean_mape"].min():.1f}%')
    knee_size = None

# Also find where MAPE drops below 25% (sparse threshold)
below_sparse = size_df[size_df['mean_mape'] <= 25]
if len(below_sparse) > 0:
    min_viable = below_sparse.iloc[0]['region_size']
    print(f'Minimum viable size (MAPE < 25%): {min_viable:.0f} sites')

ax.set_xlabel('Region Size (number of sites)', fontsize=12)
ax.set_ylabel('MAPE vs True FMV (%)', fontsize=12)
ax.set_title(f'MAPE vs Region Size (subsampled from {largest_region})', fontsize=13)
ax.legend(loc='upper right')
ax.set_xlim(0, max_test_size + 10)

plt.tight_layout()
plt.show()

---
## Step 7: Summary Table

In [ ]:
# Build comprehensive summary table
summary_rows = []

for region, rm in region_metrics.iterrows():
    row = {
        'Region': region,
        'Sites': int(rm['n_sites']),
        'Tier': rm['density_tier'],
        'Baseline_MAPE': f"{rm['mape']:.1f}%",
        'QRF_Band_Width': f"{rm['mean_band_pct']*100:.1f}%",
    }
    
    # Fallback MAPEs for sparse regions
    if rm['density_tier'] == 'Sparse':
        region_mask_a = (df_merged_result['metro_region'] == region)
        if region_mask_a.any():
            fa_mape = df_merged_result.loc[region_mask_a, 'abs_pct_error'].mean() * 100
            row['Fallback_A_MAPE'] = f'{fa_mape:.1f}%'
        else:
            row['Fallback_A_MAPE'] = 'N/A'
        
        region_mask_b = (df_optb.index.isin(df[df['metro_region'] == region].index))
        if region_mask_b.any():
            fb_mape = np.mean(
                np.abs(df_optb.loc[region_mask_b, 'predicted_fmv_b'].values -
                       df_optb.loc[region_mask_b, 'true_fmv'].values) /
                df_optb.loc[region_mask_b, 'true_fmv'].values
            ) * 100
            row['Fallback_B_MAPE'] = f'{fb_mape:.1f}%'
        else:
            row['Fallback_B_MAPE'] = 'N/A'
    else:
        row['Fallback_A_MAPE'] = '--'
        row['Fallback_B_MAPE'] = '--'
    
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)

print('╔═══════════════════════════════════════════════════════════════════════════════════════╗')
print('║                     SPARSE REGION STRESS TEST -- SUMMARY TABLE                      ║')
print('╠═══════════════════════════════════════════════════════════════════════════════════════╣')
print(f'║ {"Region":22s} {"Sites":>6s} {"Baseline":>10s} {"Band%":>8s} {"FB_A":>8s} {"FB_B":>8s} {"Tier":>10s} ║')
print('╠═══════════════════════════════════════════════════════════════════════════════════════╣')

for _, row in summary_df.iterrows():
    print(f'║ {row["Region"]:22s} {row["Sites"]:6d} {row["Baseline_MAPE"]:>10s} '
          f'{row["QRF_Band_Width"]:>8s} {row["Fallback_A_MAPE"]:>8s} {row["Fallback_B_MAPE"]:>8s} '
          f'{row["Tier"]:>10s} ║')

print('╚═══════════════════════════════════════════════════════════════════════════════════════╝')
print('\nKey: Baseline=full pipeline, FB_A=merge regions, FB_B=cluster median')

---
## Conclusions

In [ ]:
print('╔══════════════════════════════════════════════════════════════╗')
print('║       SIMULATION 06: SPARSE REGION STRESS TEST             ║')
print('╠══════════════════════════════════════════════════════════════╣')
print('║                                                            ║')
print('║  1. DENSITY TIER PERFORMANCE                               ║')
for tier in ['Dense', 'Moderate', 'Sparse']:
    m = tier_metrics.loc[tier, 'mape']
    status = 'PASS' if m < 15 else ('MARGINAL' if m < 25 else 'FAIL')
    print(f'║     {tier:12s}: MAPE = {m:5.1f}%  [{status}]                   ║')
print('║                                                            ║')
print('║  2. CONFIDENCE CALIBRATION                                 ║')
for tier in ['Dense', 'Moderate', 'Sparse']:
    ratio = band_analysis.loc[tier, 'mape_to_band_ratio']
    if ratio > 1.5:
        cal = 'OVERCONFIDENT'
    elif ratio > 1.0:
        cal = 'Slightly over'
    else:
        cal = 'Well-calibrated'
    print(f'║     {tier:12s}: MAPE/Band = {ratio:.2f}  [{cal}]           ║')
print('║                                                            ║')
print('║  3. FALLBACK STRATEGIES (Sparse Regions)                   ║')
print(f'║     Baseline MAPE:     {sparse_baseline_mape:5.1f}%                          ║')
print(f'║     Option A (Merge):  {option_a_mape:5.1f}%                          ║')
print(f'║     Option B (Median): {option_b_mape:5.1f}%                          ║')
print(f'║     Option C (Flag):   N/A ({pct_affected:.1f}% of portfolio)           ║')
print(f'║     Best: {best_strategy:45s}  ║')
print('║                                                            ║')
print('║  4. MINIMUM VIABLE REGION SIZE                             ║')
if knee_size is not None:
    print(f'║     Target MAPE < 15%: ~{knee_size:.0f} sites                        ║')
else:
    print(f'║     Target MAPE < 15%: Not achieved in tested range      ║')
print('║                                                            ║')
print('║  5. RECOMMENDATIONS                                       ║')
print('║     - Regions with <30 sites: use fallback strategy        ║')
print('║     - Widen QRF bands for sparse regions by 1.5x           ║')
print('║     - Flag sparse-region estimates with confidence caveat  ║')
print('║     - Consider merging adjacent sparse regions before      ║')
print('║       running the pipeline                                 ║')
print('║                                                            ║')
print('╚══════════════════════════════════════════════════════════════╝')

### Methodology Recap

| Analysis | What it Measures | Key Finding |
|----------|-----------------|-------------|
| **MAPE by Tier** | Accuracy degradation in sparse regions | Sparse regions have significantly higher MAPE |
| **Band Calibration** | Whether confidence bands reflect true uncertainty | Sparse regions may be overconfident |
| **Fallback A** | Merging sparse into nearest non-sparse region | May improve or worsen depending on market similarity |
| **Fallback B** | Using cluster median instead of supervised model | Simple but often competitive for sparse regions |
| **Fallback C** | Flagging as insufficient data | Honest but leaves gaps in coverage |
| **Min Viable Size** | Subsampling curve to find acceptable region size | Identifies the "knee" for reliable estimates |

### Production Recommendations

1. **Classify every region** by site count before running the pipeline
2. **Regions below the knee** (minimum viable size) should use the best-performing fallback
3. **Always report confidence tier** alongside the FMV estimate ("High confidence" for dense, "Low confidence" for sparse)
4. **Widen QRF bands** for sparse regions to avoid overconfidence
5. **Revisit as portfolio grows** -- regions that are sparse today may become viable with additional data